In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display
import time
import warnings
from selenium.webdriver.common.keys import Keys
import re

### `cate`에 넣을 값

- 김민지 : '화병·화분'
- 김예원 : '벽시계'
- 박유정 : '그림·사진'
- 송나현 : '모빌·가랜드'
- 이소현 : '장식소품'
- 이지혜 : '액자'
- 최수연 : '식물'

In [2]:
cate = '식물' ### 위의 값 넣기
xpath = '/html/body/div[1]/div/div/div[2]/div/div/div[2]/div[4]/div'
class_name = 'category-feed__content__item-wrap col-6 col-lg-4'

In [3]:
url_dic = {'화병·화분'  : 'https://ohou.se/store/category?category=2_4_3',
          '벽시계'      : 'https://ohou.se/store/category?category=2_1_0',
          '그림·사진'   : 'https://ohou.se/store/category?category=2_2_4',
          '모빌·가랜드'  : 'https://ohou.se/store/category?category=2_5_5',
          '장식소품'    : 'https://ohou.se/store/category?category=2_5_1',
          '액자'       : 'https://ohou.se/store/category?category=2_2_8',
          '식물'       : 'https://ohou.se/store/category?category=2_4_0'}

In [4]:
# 사이트 오픈
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(2)
driver.get(url_dic[cate])


# 기본 12개 url 수집
prd_area = driver.find_elements(By.CLASS_NAME,'production-item__overlay') # url 위치 선택
prd_urls = [area.get_attribute('href').split('?')[0] for area in prd_area]


# 스크롤 내리면서 나머지 url 수집
while len(prd_urls) < 101:
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    new_urls = driver.find_elements(By.CLASS_NAME,'production-item__overlay') # 새로운 url 위치 선택    
    for url in new_urls:
        url_ = url.get_attribute('href').split('?')[0]
        if url_ not in prd_urls:
            prd_urls.append(url_)
driver.close()
prd_urls = prd_urls[:100]



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\suyeo\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


---------------- 끊긴 경우 아래만 다시 돌리기 ----------------

In [5]:
# 저장된 다음 번호부터 시작. 맨 처음엔 0으로!
start = 0

In [7]:
prd_deco = pd.DataFrame(columns = ['카테고리', '상품url', '상품명', '별점', '리뷰'])
prd_deco

,카테고리,상품url,상품명,별점,리뷰


In [8]:
warnings.filterwarnings(action='ignore')
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.implicitly_wait(2)


for i in range(start, 100):
    driver.get(prd_urls[i])

    try:
        # 최신순 정렬 클릭
        driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[2]/div/div/div[1]/button[2]').click() 
        time.sleep(2)
    except:
        continue

    review_list = []
    rating_list = []

    for _ in range(20):

        reviews = driver.find_elements(By.CLASS_NAME, 'production-review-item__description') #리뷰 선택
        stars = driver.find_elements(By.CLASS_NAME, 'production-review-item__writer__info__total-star') #평점 선택

        for review, star in zip(reviews, stars) :
            star = star.get_attribute('aria-label')
            star = float(re.sub(r'[^0-9,.]', '', star))
            
            add_row = pd.DataFrame([{'카테고리' : cate,
                                     '상품url' : driver.current_url.split('?')[0], 
                                     '상품명' : driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[1]/div/div[2]/div[1]/h1/div/span').text,
                                     '별점' : star,
                                     '리뷰' : review.text.strip()}])
            prd_deco = prd_deco.append(add_row)

        try:    
            driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[5]/div/div[1]/div/section[2]/div/ul/li[11]/button').click()
            time.sleep(2)      
        except:
            break

    prd_deco.to_csv(f'prd_deco{start}_{i}.csv', index = False)
    print(i)
    
driver.close()

prd_deco.to_csv(f'prd_review_{start}_{i}.csv', index = False)



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\suyeo\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


0
1
3
4
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [10]:
prd_review_0_99 = pd.read_csv('prd_review_0_99.csv')
prd_review_0_99.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9288 entries, 0 to 9287
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   카테고리    9288 non-null   object 
 1   상품url   9288 non-null   object 
 2   상품명     9288 non-null   object 
 3   별점      9288 non-null   float64
 4   리뷰      9288 non-null   object 
dtypes: float64(1), object(4)
memory usage: 362.9+ KB
